In [1]:
import nltk
#nltk.download('stopwords')

In [2]:
#nltk.download('punkt')

In [ ]:
from werkzeug.wrappers import Request, Response
import speech_recognition as sr 
import os 
import pydub
import pyaudio
import wave
import pyttsx3
pydub.AudioSegment.converter = r"C:\\path\\to\\ffmpeg.exe"
from pydub import AudioSegment 
from pydub.silence import split_on_silence 
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
from flask import *
app = Flask(__name__)

@app.route('/')
def upload():            
    return render_template("file.html")
@app.route('/record', methods=['POST'])
def record():
    chunk = 1024  # Record in chunks of 1024 samples
    sample_format = pyaudio.paInt16  # 16 bits per sample
    channels = 2
    fs = 44100  # Record at 44100 samples per second
    seconds = 120
    filename = "output.wav"

    p = pyaudio.PyAudio()  # Create an interface to PortAudio

    print('Recording')

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []  # Initialize array to store frames

    # Store data in chunks for 3 seconds
    for i in range(0, int(fs / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)

    # Stop and close the stream 
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()

    print('Finished recording')

    # Save the recorded data as a WAV file
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    sound = AudioSegment.from_wav(filename)  
    r = sr.Recognizer()
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
    # experiment with this value for your target audio file
    min_silence_len = 600,
    # adjust this per requirement
    silence_thresh = sound.dBFS-16,
    # keep the silence for 1 second, adjustable as well
    keep_silence=400,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
            os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
    # export audio chunk and save it in
    # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            r.adjust_for_ambient_noise(source,0.3)
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    stopWords = set(stopwords.words("english")) 
    words = word_tokenize(whole_text) 
    freqTable = dict() 
    for word in words: 
        word = word.lower() 
        if word in stopWords: 
            continue
        if word in freqTable: 
            freqTable[word] += 1
        else: 
            freqTable[word] = 1
    sentences = sent_tokenize(whole_text) 
    sentenceValue = dict() 
    for sentence in sentences: 
        for word, freq in freqTable.items(): 
            if word in sentence.lower(): 
                if sentence in sentenceValue: 
                    sentenceValue[sentence] += freq 
                else: 
                    sentenceValue[sentence] = freq 
    sumValues = 0
    summary=''
    for sentence in sentenceValue: 
        sumValues += sentenceValue[sentence] 
    average = int(sumValues / len(sentenceValue)) 
    for sentence in sentences: 
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)): 
            summary += " " + sentence
    print(summary)
    text_file = open("sample.txt", "w")
    text_file.write(summary)
    return render_template("summary.html", name=summary)
@app.route('/success', methods=['POST'])
def summary():
    if request.method == 'POST':
        f = request.files['file']
        f.save(f.filename)
        sound = AudioSegment.from_wav(f.filename)  
        r = sr.Recognizer()
    # split audio sound where silence is 700 miliseconds or more and get chunks
        chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 600,
        # adjust this per requirement
        silence_thresh = sound.dBFS-16,
        # keep the silence for 1 second, adjustable as well
        keep_silence=400,
        )
        folder_name = "audio-chunks"
    # create a directory to store the audio chunks
        if not os.path.isdir(folder_name):
            os.mkdir(folder_name)
        whole_text = ""
    # process each chunk 
        for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
            chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
            audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
            with sr.AudioFile(chunk_filename) as source:
                r.adjust_for_ambient_noise(source,0.3)
                audio_listened = r.record(source)
            # try converting it to text
                try:
                    text = r.recognize_google(audio_listened)
                except sr.UnknownValueError as e:
                    print("Error:", str(e))
                else:
                    text = f"{text.capitalize()}. "
                    print(chunk_filename, ":", text)
                    whole_text += text
        stopWords = set(stopwords.words("english")) 
        words = word_tokenize(whole_text) 
        freqTable = dict() 
        for word in words: 
            word = word.lower() 
            if word in stopWords: 
                continue
            if word in freqTable: 
                freqTable[word] += 1
            else: 
                freqTable[word] = 1
        sentences = sent_tokenize(whole_text) 
        sentenceValue = dict() 
        for sentence in sentences: 
            for word, freq in freqTable.items(): 
                if word in sentence.lower(): 
                    if sentence in sentenceValue: 
                        sentenceValue[sentence] += freq 
                    else: 
                        sentenceValue[sentence] = freq 
        sumValues = 0
        summary=''
        for sentence in sentenceValue: 
            sumValues += sentenceValue[sentence] 
        average = int(sumValues / len(sentenceValue)) 
        for sentence in sentences: 
            if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)): 
                summary += " " + sentence
        text_file = open("sample.txt", "w")
        text_file.write(summary)
        return render_template("summary.html", name=summary)
@app.route('/contact', methods=['POST'])
def Audio():
    f = open("sample.txt", "r")
    ans=f.read()
    engine = pyttsx3.init()
    #get the voices of the driver
    voice=engine.getProperty('voices')
    #choose a voice and set it
    engine.setProperty('voice',voice[1].id)
    #set the volume incusive from 0.0 to 1.0
    engine.setProperty('volume',1)
    #set the rate of the speech
    engine.setProperty('rate',178)
    #summary audio is generated
    engine.say(ans)
    engine.runAndWait()
    return render_template("Audio.html")
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

C:\ProgramData\Anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/May/2021 14:01:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 14:02:17] "GET /favicon.ico HTTP/1.1" 404 -


audio-chunks\chunk1.wav : Smart contracts are very popular nowadays. 
audio-chunks\chunk2.wav : What are the problems they solve. 
audio-chunks\chunk3.wav : Smart contract was first used by nexobio in 1997 long before bitcoin was created. 
audio-chunks\chunk4.wav : Is a computer scientist lost colour and cryptographer to expire you his exact words that in simple terms you want to use a distributed ledger to store contract. 
audio-chunks\chunk5.wav : No smart contracts or just like contracts in the real world. 
audio-chunks\chunk6.wav : Difference is that they or completely digital. 
audio-chunks\chunk7.wav : In fact smart contract is actually at any computer program that is stored inside of the burgeoning. 
audio-chunks\chunk8.wav : What's the cover karen example to understand how smart contracts work. 
audio-chunks\chunk9.wav : You probably are familiar with kickstarter the largest fundraising platform. 
audio-chunks\chunk10.wav : Project team can go to text order via project sirf fun